<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6446f07c079819b276bb9a36d0be6153d7f2f5f43b332482286d94983e575f9e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 11:22:17
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 36.12 K (0.26%)
Current PnL: -27.45 L (-17.73%)
CY Booked + Current PnL: -13.16 L (-8.5%)
-------------------
Total profit:  1.42 L
Total loss:  -28.87 L
-------------------
Total Booked + Current PnL: 13.82 L (10.81%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.24%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.08 L (65.96%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,-0.06,9.59,9.22,19.69,19374.0,18393.0,210135.0,-1.08,59.0,X-MC,2.55,78.0,0.95,1.51,17.04,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.60,-20.63,26.11,0.09,20885.0,-20788.0,79989.0,85.70,25.0,M-SC,2.30,253.0,-1.00,0.58,5.09,OX40N,NTT,DURABLES
50,MASFIN,398.61,1.87,-2.53,25.21,22.05,24077.0,-2475.0,95505.0,-15.99,53.0,H-SC,6.43,164.0,-0.10,0.69,38.82,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.20,-16.11,19.22,0.02,28147.0,-28115.0,146444.0,123.21,42.0,M-SC,13.10,234.0,-1.00,1.05,27.38,OX40N,NTT,PAINTS
43,ITC,452.00,0.14,-1.73,12.73,10.78,30063.0,-4158.0,236160.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.70,3.41,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,5.42,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,71.0,L-SC,13.02,271.0,-0.05,0.65,74.42,XR,NTT,CERAMICS
72,TATAELXSI,9161.0,5.04,-24.11,73.97,32.03,74010.0,-31781.0,100054.0,-17.88,57.0,H-SC,9.64,148.0,-0.43,0.72,12.07,OX40N,NTT,IT
79,VAIBHAVGBL,521.0,2.71,-26.66,119.22,60.77,159802.0,-48735.0,134040.0,54.82,48.0,H-SC,4.87,159.0,-0.30,0.96,20.71,XR,NTT,JEWELLERY
10,BANDHANBNK,400.0,2.50,-28.34,168.53,92.42,336396.0,-78953.0,199606.0,40.09,50.0,H-SC,9.27,170.0,-0.23,1.44,15.36,XY24,NTT,BANKS
56,RAJESHEXPO,518.0,2.05,-62.75,168.64,0.07,86170.0,-86080.0,51097.0,1772.47,55.0,L-SC,2.41,268.0,-1.00,0.37,27.08,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-2.92,-38.09,254.86,119.69,427214.0,-103134.0,167627.0,-37.58,41.0,H-SC,7.50,156.0,-0.24,1.21,32.42,AR,ATH,IT
44,JCHAC,2282.00,-1.41,-36.45,57.34,-0.01,46573.0,-46583.0,81222.0,16530.43,33.0,M-SC,9.73,233.0,-1.00,0.58,3.96,OX40N,NTT,AC
61,ROUTE,2227.21,-1.20,-45.26,208.84,69.05,152111.0,-60230.0,72836.0,-55.87,57.0,H-SC,21.95,140.0,-0.40,0.52,12.33,SR,ATH,IT
32,HCLTECH,1922.01,-1.02,6.72,16.88,24.73,43582.0,16262.0,258186.0,11.49,53.0,X-LC,6.91,7.0,0.37,1.86,22.98,X40,ATH,IT
48,KPIGREEN,730.66,-1.01,-15.88,74.69,46.95,78723.0,-19898.0,105399.0,-22.21,42.0,H-SC,8.97,154.0,-0.25,0.76,32.04,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.22,-0.67,109.31,107.91,160309.0,-989.0,146655.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.06,9.35,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.22,-0.67,109.31,107.91,160309.0,-989.0,146655.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.06,9.35,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.20,-16.11,19.22,0.02,28147.0,-28115.0,146444.0,123.21,42.0,M-SC,13.10,234.0,-1.00,1.05,27.38,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.60,-20.63,26.11,0.09,20885.0,-20788.0,79989.0,85.70,25.0,M-SC,2.30,253.0,-1.00,0.58,5.09,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,5.04,-24.11,73.97,32.03,74010.0,-31781.0,100054.0,-17.88,57.0,H-SC,9.64,148.0,-0.43,0.72,12.07,OX40N,NTT,IT
66,SIS,528.00,0.08,-24.19,59.95,21.26,50464.0,-26856.0,84176.0,1986.01,50.0,H-SC,4.86,166.0,-0.53,0.61,13.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.34,8.59,58.63,72.27,107884.0,14560.0,184008.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.3,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.34,8.59,58.63,72.27,107884.0,14560.0,184008.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.30,XR,NTT,IT
25,FINCABLES,1641.55,1.22,-0.67,109.31,107.91,160309.0,-989.0,146655.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.06,9.35,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.65,-3.35,113.88,106.72,135754.0,-4128.0,119208.0,-52.15,40.0,H-SC,3.66,139.0,-0.03,0.86,19.25,AR,ATH,MISC
6,ASIANTILES,137.00,5.42,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,71.0,L-SC,13.02,271.0,-0.05,0.65,74.42,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.15,-3.51,40.57,35.63,82039.0,-7359.0,202215.0,-17.18,37.0,H-MC,2.65,119.0,-0.09,1.46,12.58,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-0.52,-11.85,45.33,28.11,80483.0,-23860.0,177550.0,-36.10,23.0,X-MC,13.81,55.0,-0.30,1.28,0.00,X40,ATH,APPARELS
8,AWL,485.00,-0.13,-24.38,102.97,53.50,234974.0,-73554.0,228197.0,-63.37,27.0,X-MC,4.23,58.0,-0.31,1.64,1.36,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.26,-18.37,90.47,55.47,179659.0,-44702.0,198584.0,-27.60,28.0,X-MC,14.37,64.0,-0.25,1.43,1.75,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.29,-39.18,122.30,35.20,95708.0,-50413.0,78257.0,2.67,31.0,X-SC,16.62,92.0,-0.53,0.56,0.38,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,0.20,-11.69,28.72,13.67,65194.0,-30063.0,227000.0,-18.28,31.0,X-LC,2.62,14.0,-0.46,1.63,7.16,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.73,-2.15,27.21,24.48,67057.0,-5412.0,246444.0,-81.32,44.0,X-SC,0.79,86.0,-0.08,1.77,20.49,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.31,-4.16,26.39,21.14,57501.0,-9449.0,217890.0,-10.87,40.0,X-MC,1.34,74.0,-0.16,1.57,22.81,XY24,NTT,PAINTS
20,DABUR,735.0,-0.14,-3.84,49.44,43.70,117938.0,-9521.0,238547.0,-11.80,33.0,X-MC,2.02,72.0,-0.08,1.72,10.63,XY24,BTT,FMCG
43,ITC,452.0,0.14,-1.73,12.73,10.78,30063.0,-4158.0,236160.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.70,3.41,X40,NTT,FMCG
78,UNITDSPR,1644.0,-0.17,2.02,18.38,20.77,43905.0,4730.0,238874.0,-6.85,39.0,X-MC,2.21,62.0,0.11,1.72,9.08,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.30,-20.88,78.86,41.51,164321.0,-54995.0,208370.0,-5.86,34.0,X-MC,7.67,61.0,-0.33,1.50,0.00,XY25,ATH,FMCG
53,PAGEIND,51605.08,-0.52,-11.85,45.33,28.11,80483.0,-23860.0,177550.0,-36.10,23.0,X-MC,13.81,55.0,-0.30,1.28,0.00,X40,ATH,APPARELS
3,ACC,3906.00,0.14,-26.15,122.27,64.15,214865.0,-62221.0,175730.0,-56.75,32.0,X-SC,3.21,82.0,-0.29,1.26,0.14,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.29,-39.18,122.30,35.20,95708.0,-50413.0,78257.0,2.67,31.0,X-SC,16.62,92.0,-0.53,0.56,0.38,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.57,-48.14,193.71,52.31,145815.0,-69885.0,75275.0,-44.83,40.0,X-SC,7.31,91.0,-0.48,0.54,0.76,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.11,-29.93,103.85,42.84,47306.0,-19454.0,45552.0,-54.27,43.0,X-SC,34.78,83.0,-0.41,0.33,3.06,XY24,NTT,MISC
59,RELAXO,1176.00,-0.57,-48.14,193.71,52.31,145815.0,-69885.0,75275.0,-44.83,40.0,X-SC,7.31,91.0,-0.48,0.54,0.76,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.29,-39.18,122.30,35.20,95708.0,-50413.0,78257.0,2.67,31.0,X-SC,16.62,92.0,-0.53,0.56,0.38,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.34,-3.44,28.49,24.08,34926.0,-4361.0,122589.0,-11.26,38.0,X-SC,5.59,89.0,-0.12,0.88,15.51,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.88,-16.70,72.04,43.32,97744.0,-27192.0,135680.0,-28.43,39.0,X-SC,5.05,90.0,-0.28,0.98,4.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.83,-9.04,32.71,20.71,112533.0,-34205.0,344032.0,-21.68,71.0,X-LC,3.86,1.0,-0.30,2.48,14.95,X40,ATH,IT
41,INFY,1972.00,1.35,12.24,19.61,34.24,68865.0,38285.0,351173.0,-12.78,70.0,X-LC,4.92,2.0,0.56,2.53,21.53,X40,NTT,IT
75,TMPV,600.00,1.63,-27.97,70.72,22.98,115076.0,-63177.0,162721.0,-25.52,40.0,X-LC,6.83,3.0,-0.55,1.17,2.34,XY24,NTT,AUTO
81,VBL,671.64,-0.94,-5.23,43.16,35.67,129186.0,-16524.0,299318.0,-16.55,48.0,X-LC,3.94,4.0,-0.13,2.15,7.90,X40N,ATH,FMCG
43,ITC,452.00,0.14,-1.73,12.73,10.78,30063.0,-4158.0,236160.0,-41.56,45.0,X-LC,2.15,5.0,-0.14,1.70,3.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,5.42,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,71.0,L-SC,13.02,271.0,-0.05,0.65,74.42,XR,NTT,CERAMICS
50,MASFIN,398.61,1.87,-2.53,25.21,22.05,24077.0,-2475.0,95505.0,-15.99,53.0,H-SC,6.43,164.0,-0.10,0.69,38.82,XR,ATH,FINANCE
13,BSOFT,831.70,-0.14,-21.54,92.19,50.79,101733.0,-30302.0,110351.0,0.50,65.0,H-SC,6.87,151.0,-0.30,0.79,28.48,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.20,-16.11,19.22,0.02,28147.0,-28115.0,146444.0,123.21,42.0,M-SC,13.10,234.0,-1.00,1.05,27.38,OX40N,NTT,PAINTS
84,WIPRO,420.00,0.34,8.59,58.63,72.27,107884.0,14560.0,184008.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.30,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.14,-21.54,92.19,50.79,101733.0,-30302.0,110351.0,0.50,65.0,H-SC,6.87,151.0,-0.30,0.79,28.48,XR,ATH,IT
6,ASIANTILES,137.00,5.42,-3.99,89.23,81.67,80366.0,-3744.0,90066.0,8111.11,71.0,L-SC,13.02,271.0,-0.05,0.65,74.42,XR,NTT,CERAMICS
84,WIPRO,420.00,0.34,8.59,58.63,72.27,107884.0,14560.0,184008.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.30,XR,NTT,IT
41,INFY,1972.00,1.35,12.24,19.61,34.24,68865.0,38285.0,351173.0,-12.78,70.0,X-LC,4.92,2.0,0.56,2.53,21.53,X40,NTT,IT
61,ROUTE,2227.21,-1.20,-45.26,208.84,69.05,152111.0,-60230.0,72836.0,-55.87,57.0,H-SC,21.95,140.0,-0.40,0.52,12.33,SR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.41
1,25,44.86
2,50,76.62


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.85
MC    30.87
LC    24.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.64
X40      23.18
X40N     12.94
XR        9.84
XY25      9.09
AR        9.00
OX40N     7.55
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
H-SC    23.71
X-LC    20.83
M-SC    11.61
X-SC     8.04
H-MC     4.81
L-SC     1.49
M-MC     1.34
M-LC     1.32
H-LC     1.12
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.26,-8.71,43.67
IT,13.21,-14.46,72.01
FINANCE,9.47,-21.15,73.84
MISC,7.26,-28.96,81.71
ELECTRICAL,6.00,-12.76,54.06
PAINTS,5.81,-14.22,31.19
INSURANCE,4.81,-1.51,36.24
PHARMA,4.14,-2.29,34.49
AUTO,2.84,-32.81,78.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3221609.0,21
AR,1324584.0,10
XR,1295188.0,13
X40,1002240.0,14
X40N,886212.0,9
OX40N,742655.0,10
XY25,377166.0,6
SR,279786.0,2
MH,78723.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3685356.0,25
M-SC,1446994.0,15
X-MC,1438654.0,16
X-LC,831398.0,11
X-SC,800828.0,8
H-MC,411986.0,3
L-SC,255029.0,3
M-LC,107884.0,1
H-LC,78622.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1296166.0      6
           AR         903830.0      5
M-SC       XY24       828014.0      6
H-SC       XR         787168.0      7
X-MC       X40        494713.0      7
           XY24       410413.0      3
           X40N       349833.0      4
H-SC       OX40N      339683.0      4
M-SC       OX40N      316802.0      5
X-LC       X40        314412.0      5
X-SC       X40N       310616.0      3
           XY24       297097.0      3
H-SC       SR         279786.0      2
X-LC       X40N       225763.0      2
H-MC       AR         216060.0      2
           XY24       195926.0      1
X-LC       XY24       193993.0      2
X-SC       X40        193115.0      2
X-MC       XY25       183695.0      2
M-SC       XR         176106.0      2
L-SC       XR         168859.0      2
M-SC       AR         126072.0      2
M-LC       XR         107884.0      1
X-LC       XY25        97230.0      2
L-SC       OX40N       86170.0      1
H-SC       MH          78723.0      1
H-LC       AR          78622.0      1
M-MC       XY25        59101.0      1
L-MC       XR          55171.0      1
L-LC       XY25        37140.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
